<h2>Assignment 3 - Big Data - Group 8<h2>
    

<b>Dataset - 311 Service Requests from 2010 to Present<br></b>
<b>Team Members</b> - <br>Amani Deepthi Matta (am10620)<br>
               Denys Fenchenko (df1911)<br>
                Mahima Mehta (mm11527)<br>

In [1]:
import gzip
import humanfriendly
import os
import pandas as pd
import numpy as np
import csv

We will be using openclean for data profiling and cleaning

In [4]:
from openclean.data.source.socrata import Socrata

In [5]:
dataset = Socrata().dataset('erm2-nwe9')

#datafile = './data/erm2-nwe9.tsv.gz'

# Remove the comment for this line if you want to use the full dataset.
datafile = './erm2-nwe9.tsv.gz'


# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)


fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

Using '311 Service Requests from 2010 to Present' in file ./erm2-nwe9.tsv.gz of size 2.91 GB


In [6]:
# Due to the size of the full dataset file, we make use of openclean's
# stream operator to avoid having to load the dataset into main-memory.

from openclean.pipeline import stream

ds_full = stream(datafile)

In [26]:
# Count number of records in the datasets.

print(f'{ds_full.count():,} rows.')


# Print the first ten rows of the dataset to get a first
# idea of the content.

ds_full.head()

27,122,579 rows.


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,34969964,12/08/2016 08:30:00 PM,12/09/2016 12:00:00 PM,DSNY,BCC - Brooklyn North,Missed Collection (All Materials),1R Missed Recycling-All Materials,Sidewalk,11212,790 ROCKAWAY AVENUE,...,,,,,,,,40.660413187950624,-73.90838198159439,"(40.660413187950624, -73.90838198159439)"
1,34969965,12/09/2016 08:14:00 AM,12/12/2016 12:00:00 PM,DSNY,BCC - Staten Island,Missed Collection (All Materials),1 Missed Collection,Sidewalk,10305,60 ALEX CIRCLE,...,,,,,,,,40.59301365304129,-74.06519740642362,"(40.59301365304129, -74.06519740642362)"
2,34969967,12/09/2016 08:53:00 AM,12/09/2016 10:30:00 AM,DOT,Department of Transportation,Traffic Signal Condition,Controller,,11234,,...,,,,,,,,40.61629487915584,-73.92989923852197,"(40.61629487915584, -73.92989923852197)"
3,34969968,12/09/2016 03:00:00 PM,12/10/2016 12:50:00 PM,DOT,Department of Transportation,Traffic Signal Condition,LED Pedestrian Unit,,11208,,...,,,,,,,,40.674384368841444,-73.87895984217538,"(40.674384368841444, -73.87895984217538)"
4,34969970,12/09/2016 06:54:15 PM,12/09/2016 06:57:55 PM,DHS,Operations Unit - Department of Homeless Services,Homeless Person Assistance,N/A,Bridge/Underpass,11211,185 MEEKER AVENUE,...,,,,,,,,40.715112875333205,-73.95230106983622,"(40.715112875333205, -73.95230106983622)"
5,34969971,12/09/2016 01:45:49 PM,12/09/2016 01:46:13 PM,TLC,Taxi and Limousine Commission,Taxi Report,Driver Report,,11369,LA GUARDIA AIRPORT,...,,,La Guardia Airport,,,,,40.77442086598845,-73.87729410513894,"(40.77442086598845, -73.87729410513894)"
6,34969973,12/09/2016 04:20:46 PM,12/09/2016 08:04:48 PM,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,10466,1890 SCHIEFFELIN AVENUE,...,,,,,,,,40.883841018857204,-73.84479382008124,"(40.883841018857204, -73.84479382008124)"
7,34969974,12/09/2016 09:00:00 AM,12/10/2016 12:00:00 PM,DSNY,BCC - Queens East,Missed Collection (All Materials),1RG Missed Recycling Paper,Sidewalk,11365,69-02 174 STREET,...,,,,,,,,40.73251928729983,-73.79588686870332,"(40.73251928729983, -73.79588686870332)"
8,34969978,12/09/2016 12:24:00 PM,12/09/2016 12:24:00 PM,DSNY,BCC - Brooklyn South,Missed Collection (All Materials),2 Bulk-Missed Collection,Sidewalk,11215,258 12 STREET,...,,,,,,,,40.66782983373874,-73.98875690143416,"(40.66782983373874, -73.98875690143416)"
9,34969979,12/09/2016 12:36:00 PM,12/10/2016 12:00:00 PM,DSNY,BCC - Staten Island,Missed Collection (All Materials),1RG Missed Recycling Paper,Sidewalk,10314,37 VILLA NOVA STREET,...,,,,,,,,40.584913088439436,-74.15879574248395,"(40.584913088439436, -74.15879574248395)"


In [8]:
# Create a view on columns in the dataset.

COLUMNS = [
    'Unique Key',
    'Created Date',
    'Closed Date',
    'Agency',
    'Agency Name',
    'Complaint Type',
    'Descriptor',
    'Location Type',
    'Incident Zip',
    'Incident Address',
    'Street Name',
    'Cross Street 1',
    'Cross Street 2',
    'Intersection Street 1',
    'Intersection Street 2',
    'Address Type',
    'City',
    'Landmark',
    'Facility Type',
    'Status',
    'Due Date',
    'Resolution Description',
    'Resolution Action Updated Date',
    'Community Board',
    'BBL',
    'Borough',
    'X Coordinate (State Plane)',
    'Y Coordinate (State Plane)',
    'Open Data Channel Type',
    'Park Facility Name',
    'Park Borough',
    'Vehicle Type',
    'Taxi Company Borough',
    'Taxi Pick Up Location',
    'Bridge Highway Name',
    'Bridge Highway Direction',
    'Road Ramp',
    'Bridge Highway Segment',
    'Latitude',
    'Longitude',
    'Location'
]

ds = ds_full.select(columns=COLUMNS)

In [9]:
type(dataset)

openclean.data.source.socrata.SODADataset

# Data Profiling

As a first step, we are profiling the data to get an insight of how the data looks.
We will be using openclean for data profiling in order to use existing libraries for getting data stats.

In [16]:
from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.profile(default_profiler=DefaultColumnProfiler)

# Print overview of profiling results.

profiles.stats()

/Users/vaibhavbavishi/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PLACE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/vaibhavbavishi/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname AVE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/vaibhavbavishi/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname ST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this wi

,total,empty,distinct,uniqueness,entropy
Unique Key,27122579,0,27122579,1.000000e+00,24.692991
Created Date,27122579,0,18740648,6.909611e-01,22.749977
Closed Date,27122579,756422,12529609,4.752156e-01,21.241846
Agency,27122579,0,32,1.179829e-06,2.947540
Agency Name,27122579,0,1933,7.126903e-05,3.334448
Complaint Type,27122579,0,473,1.743934e-05,5.921467
Descriptor,27122579,66876,1790,6.615980e-05,7.338095
Location Type,27122579,6270214,194,9.303501e-06,3.167461
Incident Zip,27122579,1424622,2826,1.099698e-04,7.213478
Incident Address,27122579,4436646,1578973,6.960141e-02,18.246697


In [20]:

# Print the most frequent data type for each column.

print('Schema\n------')
for col in ds.columns:
    p = profiles.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'Unique Key' (int)
  'Created Date' (date)
  'Closed Date' (date)
  'Agency' (str)
  'Agency Name' (str)
  'Complaint Type' (str)
  'Descriptor' (str)
  'Location Type' (str)
  'Incident Zip' (int)
  'Incident Address' (str)
  'Street Name' (str)
  'Cross Street 1' (str)
  'Cross Street 2' (str)
  'Intersection Street 1' (str)
  'Intersection Street 2' (str)
  'Address Type' (str)
  'City' (str)
  'Landmark' (str)
  'Facility Type' (str)
  'Status' (str)
  'Due Date' (date)
  'Resolution Description' (str)
  'Resolution Action Updated Date' (date)
  'Community Board' (str)
  'BBL' (int)
  'Borough' (str)
  'X Coordinate (State Plane)' (int)
  'Y Coordinate (State Plane)' (int)
  'Open Data Channel Type' (str)
  'Park Facility Name' (str)
  'Park Borough' (str)
  'Vehicle Type' (str)
  'Taxi Company Borough' (str)
  'Taxi Pick Up Location' (str)
  'Bridge Highway Name' (str)
  'Bridge Highway Direction' (str)
  'Road Ramp' (str)
  'Bridge Highway Segment' (str)
  'Latitu

In [ ]:
# Print the minimum and maximum value for column 'Created Date'

profiles.minmax('Created Date')

In [ ]:
# Print the minimum and maximum value for column 'Vehicle Year'

profiles.minmax('Closed Date')

In [ ]:
# Print the most frequent values in column 'Ticket Time'

profiles.column('Created Date').split(" ",1).get('topValues')


<h2>Data Standardization</h2>

<b>Approach 1 : Key Collision Clustering</b>

openclean provides functionality for grouping values based on similarity. This functionality is adopted from OpenRefine Clustering. The main idea is to identify clusters of values that are different but might be alternative representations of the same thing.

One clustering algorithm that is included in openclean is key collision clustering. The main idea of key collision methods is to create an alternative representation for each value (i.e., a key), and then group values based on their keys. The default key generator on openclean is the fingerprint that was adopted from OpenRefine. The main steps in creating a fingerprint key value are:

- remove leading and trailing whitespace,
- convert string to lower case,
- Normalize string by removing punctuation and control characters and replacing non-diacritic characters (if the default normalizer is used),
- Tokenize string by splitting on whitespace characters,
- Sort the tokens and remove duplicates,
- Concatenate remaining (sorted) tokens using a single space character as the delimiter.

In [18]:
# Cluster agencies using 'Key Collision' clustering with the
# default fingerprint key generator.

from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

agencies = stream(datafile).update('Agency', str.upper).distinct('Agency')
clusters = KeyCollision(func=Fingerprint()).clusters(agencies)

In [16]:
# Define simple helper method to print the k largest clusters.

def print_k_clusters(clusters, k=5):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(min(k, len(clusters))):
        print('\nCluster {}'.format(i + 1))
        for key, cnt in clusters[i].items():
            if key == '':
                key = "''"
            print(f'  {key} (x {cnt})')

In [17]:
print_k_clusters(clusters)

Total number of clusters is 0 with 0 values


<h6> Finding 1 </h6>
Since all the values in Agency column have similar format, as a result there are no clusters formed.
Hence we can say that data is already standardized for agency column

**Standardization of Street Names**

In [22]:
# Cluster street names using 'Key Collision' clustering with the
# default fingerprint key generator.

from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

street_names = stream(datafile).update('Street Name', str.upper).distinct('Street Name')
clusters = KeyCollision(func=Fingerprint()).clusters(street_names)

In [13]:
# Define simple helper method to print the k largest clusters.

def print_k_clusters(clusters, k=5):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(min(k, len(clusters))):
        print('\nCluster {}'.format(i + 1))
        for key, cnt in clusters[i].items():
            if key == '':
                key = "''"
            print(f'  {key} (x {cnt})')

In [14]:
print_k_clusters(clusters)

Total number of clusters is 2350 with 4787 values

Cluster 1
  '' (x 4437102)
  * (x 10)
  . (x 6)
  ---------- (x 1)
  ? (x 1)
  ??? (x 1)
  / (x 1)

Cluster 2
  BREWSTER RD (x 163)
  BREWSTER  RD (x 1)
  BREWSTER RD. (x 10)
  BREWSTER RD, (x 2)
  BREWSTER RD., (x 1)

Cluster 3
  N/A (x 932)
  NA (x 243)
  N\/A (x 1)
  NA/ (x 2)
  N/A/ (x 1)

Cluster 4
  3 BREWSTER ROAD (x 107)
  3 BREWSTER . ROAD (x 1)
  3 BREWSTER ROAD, (x 1)
  BREWSTER ROAD 3 (x 1)

Cluster 5
  FOREST AVE (x 172)
  FOREST AVE \ (x 1)
  FOREST AVE. (x 1)
  FOREST  AVE (x 1)


<h6>Finding 2</h6> 
Since there are different ways to write description of a particular street, so stanadarization on Street Name column forms many clusters. We are just printing 5 clusters

<h6>Specialized Key Generators</h6>

The example shows that the default fingerprint key generator already provides some promising results. The method, however, misses many cases that we found are common in U.S. street address columns. A few examples are:

Different abbreviations for street types, e.g., 125 St vs. 125 Str vs. 125 Street.
Missing whitespace between street number and street type, e.g, 125St vs. 125 St.
Text representations of street numbers, e.g., Fifth Ave vs. 5th Ave vs. 5 Ave.
To address these issues, the geospatial extension package for openclean contains a specialized key generator and value standardizer that are demonstrated in the following. Take a look at the openclean.function.token.base.Tokens class, for example, if you want to create your own customized key generator.

The following code uses the custom USStreetNameKey key generator and shows some of the values in the generated clusters. The generated clusters in general are significantly larger than the clusters that were generated using the default fingerprint algorithm. This shows one of the advantages of openclean in being customizable using domain specific functionality.

In [23]:

# Use a key generator that was specifically designed for street names.

from openclean.cluster.key import KeyCollision
from openclean_geo.address.usstreet import USStreetNameKey

# In this example we take a different approach: we first extract the list of
# distinct street names from the data file. We the apply transformations and
# clustering directly on the list of names using three parallel threads.

clusters = KeyCollision(func=USStreetNameKey(), threads=3).clusters(street_names)
print_k_clusters(clusters, k=5)

Total number of clusters is 6823 with 18055 values

Cluster 1
  9 AVENUE (x 38685)
  NINTH AVENUE (x 1672)
  9TH AVENUE (x 190)
  9TH AVE (x 60)
  9 AVE (x 140)
  9TH  AVE (x 1)
  NINTH AVE (x 2)
  9TH AV (x 1)
  9 AV (x 3)
  9AVE (x 1)
  NINTH AVE. (x 1)
  9 TH AVE (x 1)
  9TH  AV (x 1)

Cluster 2
  3 AVENUE (x 96428)
  THIRD AVENUE (x 9594)
  3RD AVENUE (x 840)
  3 AVE (x 393)
  3RD AVE (x 187)
  THIRD AVE (x 9)
  3 AV (x 8)
  3 AVE. (x 1)
  3 RD AVE (x 2)
  3AV (x 1)
  3RD AV (x 1)
  3  AVENUE (x 1)

Cluster 3
  1 AVENUE (x 63085)
  1ST AVENUE (x 345)
  FIRST AVENUE (x 6696)
  1ST AVE (x 300)
  1 AVE (x 284)
  1ST AV (x 1)
  FIRST AVE (x 5)
  1 AV (x 2)
  1AVE (x 2)
  1ST  AVE (x 1)
  1  AVENUE (x 1)

Cluster 4
  5 AVENUE (x 100632)
  5TH AVENUE (x 653)
  FIFTH AVENUE (x 8426)
  5 TH AVE (x 1)
  5 AVE (x 485)
  5TH AVE (x 266)
  FIFTH AVE (x 4)
  5 AV (x 5)
  5AV (x 1)
  5AVENUE (x 2)
  5AVE (x 2)

Cluster 5
  EAST 17 STREET (x 14734)
  EAST   17 STREET (x 13554)
  EAST 17TH STREET 

In [ ]:
# converting data pipeline to dataframe for easy data manipulation
#df = ds_full.to_df()

**Move this cell up**

<h5>City name misspelling detection</h5>
Profiling possible city misspellings in our dataset

In [27]:
CITY_COL = 'City'

In [29]:
# Use a random sample of 10,000 records for this example.

from openclean.pipeline import stream
import time

start = time.time()
#df = stream(datafile).select(CITY_COL).update(CITY_COL, str.upper).sample(10, random_state=42).to_df()
#city_df = stream(datafile).select(CITY_COL).update(CITY_COL, str.upper).to_df()
end = time.time()
print("The execution time is ", end - start, " seconds")

The execution time is  4.291534423828125e-05  seconds


In [ ]:
# Print (a subset of) the distinct city names in the sample.

city_df[CITY_COL].value_counts()

In [ ]:
# Use a counter to maintain count of how many anomaly detection operators
# classified each value as an outlier.

from collections import Counter

ensemble = Counter()

In [ ]:
# Apply fife different anomaly detection operators to the values in the city column.
# Here we use a default value embedding that ignores the frequency of each value (since
# in this NYC Open Dataset city names like NEW YORK and any of the five boroughs are
# more frequent that other names).

from openclean.embedding.feature.default import UniqueSetEmbedding
from openclean.profiling.anomalies.sklearn import (
    dbscan,
    isolation_forest,
    local_outlier_factor,
    one_class_svm,
    robust_covariance
)

for f in [dbscan, isolation_forest, local_outlier_factor, one_class_svm, robust_covariance]:
    ensemble.update(f(city_df, CITY_COL, features=UniqueSetEmbedding()))

In [ ]:
# Output values that have been classified as outliers by at least three out of the
# five operators.

prev = 0
for value, count in ensemble.most_common():
    if count < 3:
        break
    if count < prev:
        print()
    if count != prev:
        print('{}\t{}'.format(count, value))
    else:
        print('\t{}'.format(value))
    prev = count

**Testing Misspelling algorithm on top 10 rows**

In [ ]:
NUM_ROWS = 10000

In [38]:
df100k = ds_full.head(NUM_ROWS)

In [30]:
dstop10 = ds_full.head()
print(dstop10['Borough'])
new_data = dstop10['Borough'][1]
dstop10['Borough'][1] = 'BRUKLYN'
print("///////")
print(dstop10['Borough'])

0         BROOKLYN
1    STATEN ISLAND
2         BROOKLYN
3         BROOKLYN
4         BROOKLYN
5           QUEENS
6            BRONX
7           QUEENS
8         BROOKLYN
9    STATEN ISLAND
Name: Borough, dtype: object
///////
0         BROOKLYN
1          BRUKLYN
2         BROOKLYN
3         BROOKLYN
4         BROOKLYN
5           QUEENS
6            BRONX
7           QUEENS
8         BROOKLYN
9    STATEN ISLAND
Name: Borough, dtype: object


In [ ]:
from openclean.operator.transform.update import update
from openclean.function.value.null import is_empty

updated_misspelled = update(df100k, 'Borough', lambda x: 'Unknown' if is_empty(x) else x)

updated_misspelled['Borough'].value_counts()

In [32]:
from openclean.operator.transform.filter import filter
from openclean.function.eval.null import IsNotEmpty

misspelled_data = filter(df100k, predicate=IsNotEmpty('Borough'))

misspelled_data['Borough'].value_counts()

BROOKLYN         5
QUEENS           2
BRUKLYN          1
BRONX            1
STATEN ISLAND    1
Name: Borough, dtype: int64

In [34]:
from openclean.function.matching.base import DefaultStringMatcher
from openclean.function.matching.fuzzy import FuzzySimilarity
from openclean.data.mapping import Mapping

VOCABULARY = ['BROOKLYN' ,'MANHATTAN','STATEN ISLAND','BRONX', 'QUEENS']

matcher = DefaultStringMatcher(
    vocabulary=VOCABULARY,
    similarity=FuzzySimilarity()
)

map = Mapping()
for query in set(df100k['Borough']):
    map.add(query, matcher.find_matches(query))

print(map)

Mapping(<class 'list'>, {'BRONX': [StringMatch(term='BRONX', score=1)], 'STATEN ISLAND': [StringMatch(term='STATEN ISLAND', score=1)], 'BRUKLYN': [StringMatch(term='BROOKLYN', score=0.75)], 'QUEENS': [StringMatch(term='QUEENS', score=1)], 'BROOKLYN': [StringMatch(term='BROOKLYN', score=1)]})


In [35]:
from openclean.function.eval.domain import Lookup
from openclean.operator.transform.update import update
from openclean.function.eval.base import Col


# fixed = update(misspelled_data, 'Borough', Lookup(columns=['Borough'], mapping=map.to_lookup(), default=Col('Borough')))
df100k = update(df100k, 'Borough', Lookup(columns=['Borough'], mapping=map.to_lookup(), default=Col('Borough')))
print(df100k['Borough'].unique())

['BROOKLYN' 'QUEENS' 'BRONX' 'STATEN ISLAND']


**Missing values**

Standardizing empty values in Borough column to Unkonwn.


In [ ]:
df100k[‘Borough’].value_counts()

We see that there are empty values in Borough column. Hence, we update it using lambda function to standardize it.

In [ ]:
from openclean.operator.transform.update import update
from openclean.function.value.null import is_empty

check_unknown_boroughs = update(df100k, 'Borough', lambda x: 'Unknown' if is_empty(x) else x)

check_unknown_boroughs['Borough'].value_counts()

<h6>Findings</h6>
We see that there are empty values in Borough column, and our code has replaced these missing values with Unknown.


**Functional Dependencies**

In [ ]:
from openclean.operator.map.violations import fd_violations
from openclean.operator.collector.count import distinct

fd1_violations = fd_violations(df100k, [‘Longitude’, ‘Latitude'])

print('# of violations for FD(Longitude -> Latitude) is {}\n'.format(len(fd1_violations)))
for key, gr in fd1_violations.items():
    print(gr[[‘Longitude, 'Latitude']])